In [34]:
import numpy as np
import pandas as pd
import adaboost
import utils
import noising
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

%matplotlib inline

data = pd.read_csv("DataSet/Chronic_Kidney_Disease/chronic_kidney.csv", header=None)
Y_index = 1
indexes_to_add_feature_noise = [3, 4, 5, 11, 12, 13, 14, 15, 16, 17, 18, 19]
max_stds = data[indexes_to_add_feature_noise].describe().ix["std"]
pPos, pNeg = 0, 0
number_of_tries = 10

Ts = [10 * i for i in range(1, 21)]
Cs = [10 ** i for i in range(-2, 3)]

data.describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,90.000000,90.000000,90.000000,90.000000,90.00000,90.000000,90.000000,90.000000,90.000000,90.000000,...,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.477778,50.555556,77.000000,1.017778,1.40000,0.444444,0.200000,0.322222,0.155556,0.133333,...,4.960000,12.524444,37.855556,9134.444444,4.495556,0.377778,0.311111,0.122222,0.211111,0.222222
std,0.502304,14.419954,11.940866,0.006186,1.63391,1.039783,0.402241,0.469946,0.364464,0.341839,...,4.558257,3.171159,9.738792,3739.108862,1.080461,0.487548,0.465542,0.329377,0.410383,0.418069
min,0.000000,6.000000,50.000000,1.005000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.500000,3.100000,9.000000,3800.000000,2.100000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,42.250000,70.000000,1.010000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.900000,9.925000,30.000000,6725.000000,3.700000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,52.000000,80.000000,1.020000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.600000,13.200000,41.000000,8600.000000,4.550000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,59.750000,80.000000,1.025000,3.00000,0.000000,0.000000,1.000000,0.000000,0.000000,...,5.000000,15.000000,45.000000,10475.000000,5.200000,1.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,83.000000,110.000000,1.025000,4.00000,5.000000,1.000000,1.000000,1.000000,1.000000,...,47.000000,17.300000,52.000000,26400.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
def TryOuts(indexes_to_add_feature_noise, max_stds, pPos, pNeg):
    reload(noising)
    reload(utils)

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T(X_train, y_train, Ts, None, None, 0, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T : {1}, train error : {2}, val. error : {3}".format(p + 1, best_T, 1 - cross_validate_accuracy_training, error_val)

    print("Adaboost Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_C, best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T_C(X_train, y_train, Ts, 2, Cs, 0, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T, 2, best_C, 0)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T / C : {1}, {2}, train error : {3}, val. error : {4}".format(p + 1, best_T, best_C, 1 - cross_validate_accuracy_training, error_val)

    print("L1 regulariazed Adaboost (paper) Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_C, best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T_C(X_train, y_train, Ts, 2, Cs, 1, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T, 2, best_C, 1)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T : {1}, train error : {2}, val. error : {3}".format(p + 1, best_T, 1 - cross_validate_accuracy_training, error_val)

    print("Adaboost v1 Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_C, best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T_C(X_train, y_train, Ts, 3, Cs, 1, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T, 3, best_C, 1)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T : {1}, train error : {2}, val. error : {3}".format(p + 1, best_T, 1 - cross_validate_accuracy_training, error_val)

    print("Adaboost v1 pow 3Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

In [36]:
pPosMax, pNegMax = 0, 0
reload(utils)
reload(adaboost)

print "Noise level : 0 % of maximum"
noiseLevel = 0
TryOuts(indexes_to_add_feature_noise, max_stds * noiseLevel, pPosMax * noiseLevel, pNegMax * noiseLevel)

print "Noise level : 50 % of maximum"
noiseLevel = 0.5
TryOuts(indexes_to_add_feature_noise, max_stds * noiseLevel, pPosMax * noiseLevel, pNegMax * noiseLevel)

print "Noise level : 100 % of maximum"
noiseLevel = 1
TryOuts(indexes_to_add_feature_noise, max_stds * noiseLevel, pPosMax * noiseLevel, pNegMax * noiseLevel)


Noise level : 0 % of maximum
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
 The cross validation is not effective, best T found on border
Adaboost Avg. val error : 0.0, Std. : 0.0
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is no